In [ ]:
import numpy as np
from transformers import BertTokenizer, AutoModel
import torch
import datasets
import os
import argparse
from datasets import PoemDataset
from model.gpt2 import GPT2Model, GPT2Config

In [5]:
t1 = np.load("data/clear_data.npy")
t2 = np.load("data/hidden_poems.npy")
print(t1[0])
print(t2[0])
def get_tokenizer(args):
    if not os.path.exists(args.tokenizer_dir):
        os.makedirs(args.tokenizer_dir, exist_ok=True)
        BertTokenizer.from_pretrained(args.model_name).save_pretrained(args.tokenizer_dir)
    tokenizer = BertTokenizer.from_pretrained(args.tokenizer_dir, local_files_only=True)
    return tokenizer

度门能不访，冒雪屡西东。已想人如玉，遥怜马似骢。乍迷金谷路，稍变上阳宫。还比相思意，纷纷正满空。
5度冒已遥乍稍还纷->度门能不访，冒雪屡西东。已想人如玉，遥怜马似骢。乍迷金谷路，稍变上阳宫。还比相思意，纷纷正满空。


In [16]:
poem_dataset = PoemDataset("data/hidden_poems.npy", BertTokenizer.from_pretrained("cache/bert-tokenizer", local_files_only=True))
tokenizer = BertTokenizer.from_pretrained("cache/bert-tokenizer", local_files_only=True)

In [11]:
poem_dataloader = torch.utils.data.DataLoader(poem_dataset, shuffle=True, batch_size=32, collate_fn=poem_dataset.collate_fn)

In [22]:
t = next(iter(poem_dataloader))['token_ids']

In [26]:
"".join(tokenizer.convert_ids_to_tokens(t[2]))

'[CLS]7树旧鼠草迹罪伤犹->树色川光向晚晴，旧曾游处事分明。鼠穿月榭荆榛合，草掩花园畦[UNK]平。迹陷黄沙仍未[UNK]，罪标青简竟何名。伤心谷口东流水，犹喷当时寒玉声。[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [ ]:
torch.zeros(3, 5)

In [ ]:
for name in our_model.named_children():
    print(name)

In [4]:
# Test the LoRA functionality to diagnose the issue
from modules.lora_linear import LoRALinear

# Create test tensors with shapes from our error case
in_features = 768
out_features = 2304
batch_size = 8
seq_len = 86

# Test LoRALinear with use_lora=True
lora_layer = LoRALinear(in_features, out_features, r=4, use_lora=True)
input_tensor = torch.randn(batch_size, seq_len, in_features)

# Check shapes before forward pass
print(f"Input tensor shape: {input_tensor.shape}")
print(f"Linear weight shape: {lora_layer.linear.weight.shape}")
print(f"LoRA A shape: {lora_layer.lora_A.shape}")
print(f"LoRA B shape: {lora_layer.lora_B.shape}")

# Run forward pass
output = lora_layer(input_tensor)
print(f"Output shape: {output.shape}")

# Verify shapes manually
linear_output = lora_layer.linear(input_tensor)
print(f"Linear output shape: {linear_output.shape}")

lora_A_proj = torch.matmul(input_tensor, lora_layer.lora_A.T)
print(f"lora_A_proj shape: {lora_A_proj.shape}")

lora_output = torch.matmul(lora_A_proj, lora_layer.lora_B.T) * lora_layer.scaling
print(f"lora_output shape: {lora_output.shape}")

Input tensor shape: torch.Size([8, 86, 768])
Linear weight shape: torch.Size([2304, 768])
LoRA A shape: torch.Size([4, 768])
LoRA B shape: torch.Size([2304, 4])
Output shape: torch.Size([8, 86, 2304])
Linear output shape: torch.Size([8, 86, 2304])
lora_A_proj shape: torch.Size([8, 86, 4])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (688x4 and 768x4)